# **HW1: Regression** 
In *assignment 1*, you need to finish:

1.  Basic Part: Implement the regression model to predict the number of dengue cases


> *   Step 1: Split Data
> *   Step 2: Preprocess Data
> *   Step 3: Implement Regression
> *   Step 4: Make Prediction
> *   Step 5: Train Model and Generate Result

2.  Advanced Part: Implementing a regression model to predict the number of dengue cases in a different way than the basic part

# 1. Basic Part (60%)
In the first part, you need to implement the regression to predict the number of dengue cases

Please save the prediction result in a csv file **hw1_basic.csv**


## Import Packages

> Note: You **cannot** import any other package in the basic part

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
import random

## Global attributes
Define the global attributes

In [2]:
input_dataroot = 'hw1_basic_input.csv' # Input file named as 'hw1_basic_input.csv'
output_dataroot = 'hw1_basic.csv' # Output file will be named as 'hw1_basic.csv'

input_datalist =  [] # Initial datalist, saved as numpy array
output_datalist =  [] # Your prediction, should be 10 * 4 matrix and saved as numpy array
             # The format of each row should be ['epiweek', 'CityA', 'CityB', 'CityC']

You can add your own global attributes here


## Load the Input File
First, load the basic input file **hw1_basic_input.csv**

Input data would be stored in *input_datalist*

In [3]:
# Read input csv to datalist
with open(input_dataroot, newline='') as csvfile:
  input_datalist = np.array(list(csv.reader(csvfile)))
df = pd.DataFrame(input_datalist)
df = df.rename(columns={0:'epiweek',1:'TemperatureA',2:'TemperatureB',3:'TemperatureC',4:'CityA',5:'CityB',6:'CityC'})


## Implement the Regression Model

> Note: It is recommended to use the functions we defined, you can also define your own functions


### Step 1: Split Data
Split data in *input_datalist* into training dataset and validation dataset 



In [4]:
t=df.iloc[1:95,:]
v=df.iloc[95:,:]
f=df.iloc[95:,:]

full=df.iloc[1:95,:]

#training
TemperatureA=t["TemperatureA"] 
CityA=t["CityA"]
TemperatureB=t["TemperatureB"]
CityB=t["CityB"]
TemperatureC=t["TemperatureC"]
CityC=t["CityC"]


TemperatureA = TemperatureA.apply(pd.to_numeric)
TemperatureB = TemperatureB.apply(pd.to_numeric)
TemperatureC = TemperatureC.apply(pd.to_numeric)
CityA = CityA.apply(pd.to_numeric)
CityB = CityB.apply(pd.to_numeric)
CityC = CityC.apply(pd.to_numeric)

TemperatureA = TemperatureA.fillna(method='bfill')
TemperatureB = TemperatureB.fillna(method='bfill')
TemperatureC = TemperatureC.fillna(method='bfill')

#validation
valid_TemperatureA=v["TemperatureA"]
valid_CityA=v["CityA"]
valid_TemperatureB=v["TemperatureB"]
valid_CityB=v["CityB"]
valid_TemperatureC=v["TemperatureC"]
valid_CityC=v["CityC"]

valid_TemperatureA = valid_TemperatureA.apply(pd.to_numeric).reset_index(drop=True)
valid_TemperatureB = valid_TemperatureB.apply(pd.to_numeric).reset_index(drop=True)
valid_TemperatureC = valid_TemperatureC.apply(pd.to_numeric).reset_index(drop=True)
valid_CityA = valid_CityA.apply(pd.to_numeric).reset_index(drop=True)
valid_CityB = valid_CityB.apply(pd.to_numeric).reset_index(drop=True)
valid_CityC = valid_CityC.apply(pd.to_numeric).reset_index(drop=True)

valid_TemperatureA = valid_TemperatureA.fillna(method='bfill')
valid_TemperatureB = valid_TemperatureB.fillna(method='bfill')
valid_TemperatureC = valid_TemperatureC.fillna(method='bfill')

### Step 2: Preprocess Data
Handle the unreasonable data
> Hint: Outlier and missing data can be handled by removing the data or adding the values with the help of statistics  

In [5]:
def outlier(df,col,n):
    mean = df[col].mean()
    sd = df[col].std()
    df = df[(df[col] <= mean+(n*sd))]
    df = df[(df[col] >= mean-(n*sd))]
    return df

In [6]:
CityAshift1= CityA.shift(periods=1)
CityAshift1= CityAshift1.fillna(method='bfill')
CityAshift2= CityAshift1.shift(periods=1)
CityAshift2= CityAshift2.fillna(method='bfill')
CityAshift3= CityAshift2.shift(periods=1)
CityAshift3= CityAshift3.fillna(method='bfill')

CityBshift1= CityB.shift(periods=1)
CityBshift1= CityBshift1.fillna(method='bfill')
CityBshift2= CityBshift1.shift(periods=1)
CityBshift2= CityBshift2.fillna(method='bfill')
CityBshift3= CityBshift2.shift(periods=1)
CityBshift3= CityBshift3.fillna(method='bfill')

CityCshift1= CityC.shift(periods=1)
CityCshift1= CityCshift1.fillna(method='bfill')
CityCshift2= CityCshift1.shift(periods=1)
CityCshift2= CityCshift2.fillna(method='bfill')
CityCshift3= CityCshift2.shift(periods=1)
CityCshift3= CityCshift3.fillna(method='bfill')



# traindataA=pd.DataFrame({0:TemperatureA, 1:CityAshift1, 2:CityAshift2,3:CityA })
traindataA=pd.DataFrame({0:TemperatureA, 1:CityAshift1, 2:CityAshift2,3:CityAshift3, 4:CityA })
traindataA=outlier(traindataA,0,1.5)
# traindataA=outlier(traindataA,1,2.5)
# traindataA=outlier(traindataA,2,2.5)
# traindataA=outlier(traindataA,3,2.5)
CityA_fortrain = traindataA.pop(4)
# CityA_fortrain = traindataA.pop(3)

# traindataB=pd.DataFrame({0:TemperatureB, 1:CityBshift1, 2:CityBshift2,3:CityB })
traindataB=pd.DataFrame({0:TemperatureB, 1:CityBshift1, 2:CityBshift2,3:CityBshift3, 4:CityB })
traindataB=outlier(traindataB,0,1.5)
# traindataB=outlier(traindataB,1,2.5)
# traindataB=outlier(traindataB,2,2.5)
# traindataB=outlier(traindataB,3,2.5)
CityB_fortrain = traindataB.pop(4)
# CityB_fortrain = traindataB.pop(3)

# traindataC=pd.DataFrame({0:TemperatureC, 1:CityCshift1, 2:CityCshift2,3:CityC })
traindataC=pd.DataFrame({0:TemperatureC, 1:CityCshift1, 2:CityCshift2,3:CityCshift3, 4:CityC })
traindataC=outlier(traindataC,0,1.5)
# traindataC=outlier(traindataC,1,2.5)
# traindataC=outlier(traindataC,2,2.5)
# traindataC=outlier(traindataC,3,2.5)
CityC_fortrain = traindataC.pop(4)
# CityC_fortrain = traindataC.pop(3)

order_A=[]
order_A.append(1)
order_A.append(1)
order_A.append(1)
order_A.append(1)

order_B=[]
order_B.append(1)
order_B.append(1)
order_B.append(1)
order_B.append(1)

order_C=[]
order_C.append(1)
order_C.append(1)
order_C.append(1)
order_C.append(1)


lastrow_traindataA = traindataA.iloc[-1:]
lastrow_traindataA = lastrow_traindataA.reset_index(drop=True)
# variables_valid1_A = pd.DataFrame({0:valid_TemperatureA.iloc[0], 1:lastrow_traindataA[1], 2:lastrow_traindataA[2] })
variables_valid1_A = pd.DataFrame({0:valid_TemperatureA.iloc[0], 1:lastrow_traindataA[1], 2:lastrow_traindataA[2] ,3:lastrow_traindataA[3] })

lastrow_traindataB = traindataB.iloc[-1:]
lastrow_traindataB = lastrow_traindataB.reset_index(drop=True)
# variables_valid1_B = pd.DataFrame({0:valid_TemperatureB.iloc[0], 1:lastrow_traindataB[1], 2:lastrow_traindataB[2] })
variables_valid1_B = pd.DataFrame({0:valid_TemperatureB.iloc[0], 1:lastrow_traindataB[1], 2:lastrow_traindataB[2] ,3:lastrow_traindataB[3]})

lastrow_traindataC = traindataC.iloc[-1:]
lastrow_traindataC = lastrow_traindataC.reset_index(drop=True)
# variables_valid1_C = pd.DataFrame({0:valid_TemperatureC.iloc[0], 1:lastrow_traindataC[1], 2:lastrow_traindataC[2] })
variables_valid1_C = pd.DataFrame({0:valid_TemperatureC.iloc[0], 1:lastrow_traindataC[1], 2:lastrow_traindataC[2],3:lastrow_traindataC[3] })

In [7]:
def mkmatrix(x,order):    
    for i in range(len(order)):
        x[i]= x[i].pow(order[i])

    (n, p_minus_one) = x.shape
    p = p_minus_one + 1
    new_x = np.ones(shape=(n, p))
    new_x[:, 1:] = x

    # print(new_x)
    return new_x

### Step 3: Implement Regression
> Hint: You can use Matrix Inversion, or Gradient Descent to finish this part




In [8]:
def Regression(x, y):  
  coeff = np.dot(np.dot(np.linalg.inv(np.dot(x.T, x)), x.T), y)
  return coeff

### Step 4: Make Prediction
Make prediction of testing dataset and store the value in *output_datalist*

In [9]:
def MakePrediction(valid,trained_coeff,valid_Temperature,order):
  for i in range(3,10):
    tmp=valid[i-1]
    tmp2=valid[i-2]
    tmp3=valid[i-3]
    variables_valid = pd.DataFrame({0:valid_Temperature.iloc[i], 1:tmp, 2:tmp2,3:tmp3})
    # variables_valid = pd.DataFrame({0:valid_Temperature.iloc[i], 1:tmp, 2:tmp2})
    valid.append(np.dot(mkmatrix(variables_valid,order), trained_coeff))
  predicted= []
  for i in range(len(valid)):
    predicted.append(valid[i][0])
  return(predicted)
    

### Step 5: Train Model and Generate Result

> Notice: **Remember to output the coefficients of the model here**, otherwise 5 points would be deducted
* If your regression model is *3x^2 + 2x^1 + 1*, your output would be: 
```
3 2 1
```





In [10]:
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    # for i in range(len(actual)):
    #     print(actual[i],pred[i])
    # print("--------------------------------")
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [11]:
trained_coeff_A= Regression(mkmatrix(traindataA,order_A),CityA_fortrain)
trained_coeff_B= Regression(mkmatrix(traindataB,order_B),CityB_fortrain)
trained_coeff_C= Regression(mkmatrix(traindataC,order_C),CityC_fortrain)

tmparr=np.array([trained_coeff_A,trained_coeff_B,trained_coeff_C])
coeff_basic=pd.DataFrame(tmparr, index = ['A','B','C'],columns=['W[0]', 'W[1]', 'W[2]', 'W[3]', 'W[4]'])
print(coeff_basic)
print("function: y[i] = w[0] + w[1]*Temperature[i] + w[2]*y[i-1] + w[3]*y[i-2] + w[4]*y[i-3]")

valid_A=[]
valid_A.append(np.dot(mkmatrix(variables_valid1_A,order_A), trained_coeff_A))
variables_valid2_A = pd.DataFrame({0:valid_TemperatureB.iloc[1], 1:valid_A[0], 2:lastrow_traindataA[1],3:lastrow_traindataA[2]})
# variables_valid2_A = pd.DataFrame({0:valid_TemperatureB.iloc[1], 1:valid_A[0], 2:lastrow_traindataA[1]})
valid_A.append(np.dot(mkmatrix(variables_valid2_A,order_A), trained_coeff_A))

variables_valid3_A = pd.DataFrame({0:valid_TemperatureA.iloc[2], 1:valid_A[1], 2:valid_A[0],3:lastrow_traindataA[1]})
valid_A.append(np.dot(mkmatrix(variables_valid3_A,order_A), trained_coeff_A))

predicted_A = MakePrediction(valid_A,trained_coeff_A,valid_TemperatureA,order_A)

valid_B=[]
valid_B.append(np.dot(mkmatrix(variables_valid1_B,order_B), trained_coeff_B))
variables_valid2_B = pd.DataFrame({0:valid_TemperatureB.iloc[1], 1:valid_B[0], 2:lastrow_traindataB[1],3:lastrow_traindataB[2]})
# variables_valid2_B = pd.DataFrame({0:valid_TemperatureB.iloc[1], 1:valid_B[0], 2:lastrow_traindataB[1]})
valid_B.append(np.dot(mkmatrix(variables_valid2_B,order_B), trained_coeff_B))

variables_valid3_B = pd.DataFrame({0:valid_TemperatureB.iloc[2], 1:valid_B[1], 2:valid_B[0],3:lastrow_traindataB[1]})
valid_B.append(np.dot(mkmatrix(variables_valid3_B,order_B), trained_coeff_B))

predicted_B = MakePrediction(valid_B,trained_coeff_B,valid_TemperatureB,order_B)

valid_C=[]
valid_C.append(np.dot(mkmatrix(variables_valid1_C,order_C), trained_coeff_C))
variables_valid2_C = pd.DataFrame({0:valid_TemperatureC.iloc[1], 1:valid_C[0], 2:lastrow_traindataC[1],3:lastrow_traindataC[2]})
# variables_valid2_C = pd.DataFrame({0:valid_TemperatureC.iloc[1], 1:valid_C[0], 2:lastrow_traindataC[1]})
valid_C.append(np.dot(mkmatrix(variables_valid2_C,order_C), trained_coeff_C))

variables_valid3_C = pd.DataFrame({0:valid_TemperatureC.iloc[2], 1:valid_C[1], 2:valid_C[0],3:lastrow_traindataC[1]})
valid_C.append(np.dot(mkmatrix(variables_valid3_C,order_C), trained_coeff_C))

predicted_C = MakePrediction(valid_C,trained_coeff_C,valid_TemperatureC,order_C)


# mape_A = mape(valid_CityA,predicted_A)
# mape_B = mape(valid_CityB,predicted_B)
# mape_C = mape(valid_CityC,predicted_C)
# print("mapeA:",mape_A,"mapeB:",mape_B,"mapeC:",mape_C)

###########################################
date=f["epiweek"]
output_datalist=np.array([date,predicted_A,predicted_B,predicted_C])
output_datalist=np.transpose(output_datalist)
# print(date)


with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  for row in output_datalist:
    writer.writerow(row)
    
###########################################
# print(lastrow_traindataC)
# print(variables_valid2_C)
# print(variables_valid3_C)
# print(predicted_C)
# problemCity=full["CityA"]
# problemCity=problemCity.apply(pd.to_numeric)
# problemTemperature=full["TemperatureA"]
# problemTemperature=problemTemperature.apply(pd.to_numeric)

# print(problemCity.to_string())

# plt.subplot(2, 1, 1)
# plt.plot(problemCity)

# #plot 2:
# # plt.subplot(2, 1, 2)
# # plt.plot(valid_CityA)

# plt.subplot(2, 1, 2)
# plt.plot(predicted_A,color="red")


        W[0]      W[1]      W[2]      W[3]      W[4]
A  25.611815 -0.872774  0.662262  0.225151  0.022688
B  24.140854 -0.884669  0.460727  0.332103  0.053155
C   7.190799 -0.175476  0.896345 -0.012603  0.028962
function: y[i] = w[0] + w[1]*Temperature[i] + w[2]*y[i-1] + w[3]*y[i-2] + w[4]*y[i-3]


## Write the Output File
Write the prediction to output csv
> Format: 'epiweek', 'CityA', 'CityB', 'CityC'

# 2. Advanced Part (35%)
In the second part, you need to implement the regression in a different way than the basic part to help your predictions for the number of dengue cases

We provide you with two files **hw1_advanced_input1.csv** and **hw1_advanced_input2.csv** that can help you in this part

Please save the prediction result in a csv file **hw1_advanced.csv** 


In [12]:
def MakePrediction_advanced(valid,trained_coeff,valid_Temperature,order,valid_Rain):
  for i in range(3,10):
    tmp=valid[i-1]
    tmp2=valid[i-2]
    tmp3=valid[i-3]
    variables_valid = pd.DataFrame({0:valid_Temperature.iloc[i], 1:tmp, 2:tmp2,3:tmp3,4:valid_Rain.iloc[i]})
    # variables_valid = pd.DataFrame({0:valid_Temperature.iloc[i], 1:tmp, 2:tmp2})
    valid.append(np.dot(mkmatrix(variables_valid,order), trained_coeff))
  predicted= []
  for i in range(len(valid)):
    predicted.append(valid[i][0])
  return(predicted)
    

In [13]:
input_dataroot_advance1 = 'hw1_advanced_input1.csv' # Input file named as 'hw1_basic_input.csv'
output_dataroot_advance = 'hw1_advanced.csv' # Output file will be named as 'hw1_basic.csv'

input_datalist_advance1 =  [] # Initial datalist, saved as numpy array
output_datalist_advance =  [] # Your prediction, should be 10 * 4 matrix and saved as numpy array
             # The format of each row should be ['epiweek', 'CityA', 'CityB', 'CityC']
             # Read input csv to datalist
with open(input_dataroot_advance1, newline='') as csvfile:
  input_datalist_advance1 = np.array(list(csv.reader(csvfile)))

df1 = pd.DataFrame(input_datalist_advance1)

t_advanced=df1.iloc[1:95,:]
v_advanced=df1.iloc[95:,:]
f_advanced=df1.iloc[95:,:]
full_advanced1=df1.iloc[1:95,:]


#training
RainA=t_advanced[1]
RainB=t_advanced[2]
RainC=t_advanced[3]

RainA = RainA.apply(pd.to_numeric)
RainB = RainB.apply(pd.to_numeric)
RainC = RainC.apply(pd.to_numeric)

valid_RainA=v_advanced[1]
valid_RainB=v_advanced[2]
valid_RainC=v_advanced[3]
valid_RainA = valid_RainA.apply(pd.to_numeric).reset_index(drop=True)
valid_RainB = valid_RainB.apply(pd.to_numeric).reset_index(drop=True)
valid_RainC = valid_RainC.apply(pd.to_numeric).reset_index(drop=True)


traindataA_advanced=pd.DataFrame({0:TemperatureA, 1:CityAshift1, 2:CityAshift2,3:CityAshift3,4:RainA, 5:CityA })
traindataA_advanced=outlier(traindataA_advanced,0,1.5)
CityA_fortrain_advanced = traindataA_advanced.pop(5)

traindataB_advanced=pd.DataFrame({0:TemperatureB, 1:CityBshift1, 2:CityBshift2,3:CityBshift3,4:RainB, 5:CityB})
traindataB_advanced=outlier(traindataB_advanced,0,1.5)
CityB_fortrain_advanced = traindataB_advanced.pop(5)

traindataC_advanced=pd.DataFrame({0:TemperatureC, 1:CityCshift1, 2:CityCshift2,3:CityCshift3,4:RainC, 5:CityC})
traindataC_advanced=outlier(traindataC_advanced,0,1.5)
CityC_fortrain_advanced = traindataC_advanced.pop(5)

order_advanced_A=[]
order_advanced_A.append(1)
order_advanced_A.append(1)
order_advanced_A.append(1)
order_advanced_A.append(1)
order_advanced_A.append(2)

order_advanced_B=[]
order_advanced_B.append(1)
order_advanced_B.append(1)
order_advanced_B.append(1)
order_advanced_B.append(1)
order_advanced_B.append(2)

order_advanced_C=[]
order_advanced_C.append(1)
order_advanced_C.append(1)
order_advanced_C.append(1)
order_advanced_C.append(1)
order_advanced_C.append(2)

trained_coeff_advanced_A= Regression(mkmatrix(traindataA_advanced,order_advanced_A),CityA_fortrain_advanced)
trained_coeff_advanced_B= Regression(mkmatrix(traindataB_advanced,order_advanced_B),CityB_fortrain_advanced)
trained_coeff_advanced_C= Regression(mkmatrix(traindataC_advanced,order_advanced_C),CityC_fortrain_advanced)

tmparr=np.array([trained_coeff_advanced_A,trained_coeff_advanced_B,trained_coeff_advanced_C])
coeff_advanced=pd.DataFrame(tmparr, index = ['A','B','C'],columns=['W[0]', 'W[1]', 'W[2]', 'W[3]', 'W[4]', 'W[5]'])
print(coeff_advanced)
print("function: y[i] = w[0] + w[1]*Temperature[i] + w[2]*y[i-1] + w[3]*y[i-2] + w[4]*y[i-3] + w[5]*Precipitation[i]")

lastrow_traindataA_advanced = traindataA_advanced.iloc[-1:]
lastrow_traindataA_advanced = lastrow_traindataA_advanced.reset_index(drop=True)
variables_valid1_advanced_A = pd.DataFrame({0:valid_TemperatureA.iloc[0], 1:lastrow_traindataA_advanced[1], 
                                   2:lastrow_traindataA_advanced[2] ,3:lastrow_traindataA_advanced[3],4:valid_RainA.iloc[0] })

lastrow_traindataB_advanced = traindataB_advanced.iloc[-1:]
lastrow_traindataB_advanced = lastrow_traindataB_advanced.reset_index(drop=True)
variables_valid1_advanced_B = pd.DataFrame({0:valid_TemperatureB.iloc[0], 1:lastrow_traindataB_advanced[1],
                                   2:lastrow_traindataB_advanced[2] ,3:lastrow_traindataB_advanced[3],4:valid_RainB.iloc[0]})

lastrow_traindataC_advanced = traindataC_advanced.iloc[-1:]
lastrow_traindataC_advanced = lastrow_traindataC_advanced.reset_index(drop=True)
variables_valid1_advanced_C = pd.DataFrame({0:valid_TemperatureC.iloc[0], 1:lastrow_traindataC_advanced[1],
                                   2:lastrow_traindataC_advanced[2],3:lastrow_traindataC_advanced[3],4:valid_RainC.iloc[0] })

valid_A_advanced=[]
valid_A_advanced.append(np.dot(mkmatrix(variables_valid1_advanced_A,order_advanced_A), trained_coeff_advanced_A))
variables_valid2_advanced_A = pd.DataFrame({0:valid_TemperatureB.iloc[1], 1:valid_A_advanced[0],
                                            2:lastrow_traindataA_advanced[1],3:lastrow_traindataA_advanced[2],4:valid_RainA.iloc[1]})
valid_A_advanced.append(np.dot(mkmatrix(variables_valid2_advanced_A,order_advanced_A), trained_coeff_advanced_A))
variables_valid3_advanced_A = pd.DataFrame({0:valid_TemperatureA.iloc[2], 1:valid_A_advanced[1],
                                            2:valid_A_advanced[0],3:lastrow_traindataA_advanced[1],4:valid_RainA.iloc[2]})
valid_A_advanced.append(np.dot(mkmatrix(variables_valid3_advanced_A,order_advanced_A), trained_coeff_advanced_A))
predicted_advanced_A = MakePrediction_advanced(valid_A_advanced,trained_coeff_advanced_A,valid_TemperatureA,order_advanced_A,valid_RainA)

valid_advanced_B=[]
valid_advanced_B.append(np.dot(mkmatrix(variables_valid1_advanced_B,order_advanced_B), trained_coeff_advanced_B))
variables_valid2_advanced_B = pd.DataFrame({0:valid_TemperatureB.iloc[1], 1:valid_advanced_B[0],
                                   2:lastrow_traindataB_advanced[1],3:lastrow_traindataB_advanced[2],4:valid_RainB.iloc[1]})
valid_advanced_B.append(np.dot(mkmatrix(variables_valid2_advanced_B,order_advanced_B), trained_coeff_advanced_B))

variables_valid3_advanced_B = pd.DataFrame({0:valid_TemperatureB.iloc[2], 1:valid_advanced_B[1],
                                   2:valid_advanced_B[0],3:lastrow_traindataB_advanced[1],4:valid_RainB.iloc[2]})
valid_advanced_B.append(np.dot(mkmatrix(variables_valid3_advanced_B,order_advanced_B), trained_coeff_advanced_B))
predicted_advanced_B = MakePrediction_advanced(valid_advanced_B,trained_coeff_advanced_B,valid_TemperatureB,order_advanced_B,valid_RainB)

valid_advanced_C=[]
valid_advanced_C.append(np.dot(mkmatrix(variables_valid1_advanced_C,order_advanced_C), trained_coeff_advanced_C))
variables_valid2_advanced_C = pd.DataFrame({0:valid_TemperatureC.iloc[1], 1:valid_advanced_C[0],
                                   2:lastrow_traindataC[1],3:lastrow_traindataC[2],4:valid_RainC.iloc[1]})
valid_advanced_C.append(np.dot(mkmatrix(variables_valid2_advanced_C,order_advanced_C), trained_coeff_advanced_C))

variables_valid3_advanced_C = pd.DataFrame({0:valid_TemperatureC.iloc[2], 1:valid_advanced_C[1],
                                   2:valid_advanced_C[0],3:lastrow_traindataC[1],4:valid_RainC.iloc[2]})
valid_advanced_C.append(np.dot(mkmatrix(variables_valid3_advanced_C,order_advanced_C), trained_coeff_advanced_C))

predicted_advanced_C = MakePrediction_advanced(valid_advanced_C,trained_coeff_advanced_C,valid_TemperatureC,order_advanced_C,valid_RainC)

# mape_advanced_A = mape(valid_CityA,predicted_advanced_A)
# mape_advanced_B = mape(valid_CityB,predicted_advanced_B)
# mape_advanced_C = mape(valid_CityC,predicted_advanced_C)
# print(traindataA_advanced)
# print(trained_coeff_A)
# print("mapeA_advanced:",mape_advanced_A,"mapeB_advanced:",mape_advanced_B,"mapeC_advanced:",mape_advanced_C)

###########################################
date=f["epiweek"]
output_datalist_advanced=np.array([date,predicted_advanced_A,predicted_advanced_B,predicted_advanced_C])
output_datalist_advanced=np.transpose(output_datalist_advanced)

with open(output_dataroot_advance, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  for row in output_datalist_advanced:
    writer.writerow(row)
    
###########################################

# problemCity=full["CityA"]
# problemCity=problemCity.apply(pd.to_numeric)
# problemvarible=full_advanced1[1]
# problemvarible=problemvarible.apply(pd.to_numeric)

# plt.subplot(2, 1, 1)
# plt.plot(problemCity)
# plt.plot(problemvarible,color="red")

# #plot 2:
# plt.subplot(2, 1, 2)
# plt.plot(valid_CityC)

# plt.subplot(2, 1, 2)
# plt.plot(predicted_C,color="red")

# print(RainB)

        W[0]      W[1]      W[2]      W[3]      W[4]      W[5]
A  25.174366 -0.864571  0.661624  0.225986  0.022867  0.005427
B  24.108258 -0.884250  0.460650  0.332302  0.053207  0.000416
C   4.160512 -0.086211  0.895150 -0.072333  0.069955  0.008229
function: y[i] = w[0] + w[1]*Temperature[i] + w[2]*y[i-1] + w[3]*y[i-2] + w[4]*y[i-3] + w[5]*Precipitation[i]


# Report *(5%)*

Report should be submitted as a pdf file **hw1_report.pdf**

*   Briefly describe the difficulty you encountered 
*   Summarize your work and your reflections 
*   No more than one page






# Save the Code File
Please save your code and submit it as an ipynb file! (**hw1.ipynb**)